## EDA LUNG CANCER 

Luis Angel Garcia(2230177)

Juliana Toro ()

Antonio Cardenas Jurado(2230433)

## Libraries

In [377]:
import matplotlib.pyplot as plt
import pandas as pd
import sys

sys.path.append("../config_documents")

import postgres_setup as pg
import importlib
importlib.reload(pg)
import numpy as np
import seaborn as sns
import logging
import numpy as np

Logging config

In [378]:
# Eliminar handlers anteriores
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Configurar logging para que imprima en la celda (stdout)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

Read the dataset from CSV

In [379]:
df = pd.read_csv("../dataset/lung_cancer_prediction_dataset.csv")

Analysis of first fields

In [380]:
print("\nPrimeras filas del dataset:")
print(df.head())


Primeras filas del dataset:
   ID       Country  Population_Size  Age  Gender Smoker  Years_of_Smoking  \
0   0         China             1400   80    Male    Yes                30   
1   1          Iran               84   53    Male     No                 0   
2   2        Mexico              128   47    Male    Yes                12   
3   3     Indonesia              273   39  Female     No                 0   
4   4  South Africa               59   44  Female     No                 0   

   Cigarettes_per_Day Passive_Smoker Family_History  ...  \
0                  29             No             No  ...   
1                   0            Yes             No  ...   
2                   6            Yes             No  ...   
3                   0             No            Yes  ...   
4                   0            Yes             No  ...   

  Air_Pollution_Exposure Occupational_Exposure  Indoor_Pollution  \
0                    Low                   Yes                No   
1    

General information

In [381]:
print("\nInformación del dataset:")
logging.info(df.info())


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220632 entries, 0 to 220631
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           220632 non-null  int64  
 1   Country                      220632 non-null  object 
 2   Population_Size              220632 non-null  int64  
 3   Age                          220632 non-null  int64  
 4   Gender                       220632 non-null  object 
 5   Smoker                       220632 non-null  object 
 6   Years_of_Smoking             220632 non-null  int64  
 7   Cigarettes_per_Day           220632 non-null  int64  
 8   Passive_Smoker               220632 non-null  object 
 9   Family_History               220632 non-null  object 
 10  Lung_Cancer_Diagnosis        220632 non-null  object 
 11  Cancer_Stage                 8961 non-null    object 
 12  Survival_Years               220

null values

In [382]:
logging.info(f"cantidad de nulos:\n {df.isnull().sum()}")

2025-04-12 12:18:38,684 - INFO - cantidad de nulos:
 ID                                  0
Country                             0
Population_Size                     0
Age                                 0
Gender                              0
Smoker                              0
Years_of_Smoking                    0
Cigarettes_per_Day                  0
Passive_Smoker                      0
Family_History                      0
Lung_Cancer_Diagnosis               0
Cancer_Stage                   211671
Survival_Years                      0
Adenocarcinoma_Type                 0
Air_Pollution_Exposure              0
Occupational_Exposure               0
Indoor_Pollution                    0
Healthcare_Access                   0
Early_Detection                     0
Treatment_Type                 213968
Developed_or_Developing             0
Annual_Lung_Cancer_Deaths           0
Lung_Cancer_Prevalence_Rate         0
Mortality_Rate                      0
dtype: int64


duplicates values

In [383]:
logging.info(f"\nCantidad de filas duplicadas: {df.duplicated().sum()}")

2025-04-12 12:18:38,833 - INFO - 
Cantidad de filas duplicadas: 0


Unique values

In [384]:
print("\nValores únicos en la columna 'Gender':", df['Gender'].unique())
print("Valores únicos en la columna 'Smoker':", df['Smoker'].unique())
print("Valores únicos en la columna 'Passive_Smoker':", df['Passive_Smoker'].unique())
print("Valores únicos en la columna 'Family_History':", df['Family_History'].unique())
print("Valores únicos en la columna 'Lung_Cancer_Diagnosis':", df['Lung_Cancer_Diagnosis'].unique())
print("Valores únicos en la columna 'Cancer_Stage':", df['Cancer_Stage'].unique())
print("Valores únicos en la columna 'Adenocarcinoma_Type':", df['Adenocarcinoma_Type'].unique())
print("Valores únicos en la columna 'Air_pollution_exposure':", df['Air_Pollution_Exposure'].unique())
print("Valores únicos en la columna 'Occupational_Exposure':", df['Occupational_Exposure'].unique())
print("Valores únicos en la columna 'Indoor_pollution':", df['Indoor_Pollution'].unique())
print("Valores únicos en la columna 'Healtcare_Access':", df['Healthcare_Access'].unique())
print("Valores únicos en la columna 'Early_Detection':", df['Early_Detection'].unique())
print("Valores únicos en la columna 'Developed_or_Developing':", df['Developed_or_Developing'].unique())


Valores únicos en la columna 'Gender': ['Male' 'Female']
Valores únicos en la columna 'Smoker': ['Yes' 'No']
Valores únicos en la columna 'Passive_Smoker': ['No' 'Yes']
Valores únicos en la columna 'Family_History': ['No' 'Yes']
Valores únicos en la columna 'Lung_Cancer_Diagnosis': ['No' 'Yes']
Valores únicos en la columna 'Cancer_Stage': [nan 'Stage 1' 'Stage 2' 'Stage 3' 'Stage 4']
Valores únicos en la columna 'Adenocarcinoma_Type': ['Yes' 'No']
Valores únicos en la columna 'Air_pollution_exposure': ['Low' 'Medium' 'High']
Valores únicos en la columna 'Occupational_Exposure': ['Yes' 'No']
Valores únicos en la columna 'Indoor_pollution': ['No' 'Yes']
Valores únicos en la columna 'Healtcare_Access': ['Poor' 'Good']
Valores únicos en la columna 'Early_Detection': ['No' 'Yes']
Valores únicos en la columna 'Developed_or_Developing': ['Developing' 'Developed']


Cigarrites per day analisys

In [385]:
min_cigarettes = df["Cigarettes_per_Day"].min()
max_cigarettes = df["Cigarettes_per_Day"].max()

logging.info(f"\nLos minimos cigarrillos al dia encontrados fueron: {min_cigarettes}")
logging.info(f"Los maximos cigarrillos al dia encontrados fueron: {max_cigarettes}")

smokers = (df["Cigarettes_per_Day"] > 0).sum()
logging.info(f"\nNúmero de personas que fuman son fumadoras: {smokers}")

df_smokers = df[df["Cigarettes_per_Day"]>0]

q1 = df_smokers["Cigarettes_per_Day"].quantile(0.25)
q3 = df_smokers["Cigarettes_per_Day"].quantile(0.75)
iqr=q3-q1

limit_inf = max(1, q1- 1.5 * iqr)
limit_sup = q3 + 1.5 * iqr

atipic_value= df_smokers[(df_smokers["Cigarettes_per_Day"] < limit_inf) | (df_smokers["Cigarettes_per_Day"]>limit_sup)]

logging.info(f"\nQ1: {q1}, Q3: {q3}, IQR: {iqr}")
logging.info(f"Límite inferior ajustado: {limit_inf}")
logging.info(f"Límite superior: {limit_sup}")
logging.info(f"Número de valores atípicos: {atipic_value.shape[0]}")

2025-04-12 12:18:38,943 - INFO - 
Los minimos cigarrillos al dia encontrados fueron: 0
2025-04-12 12:18:38,944 - INFO - Los maximos cigarrillos al dia encontrados fueron: 30
2025-04-12 12:18:38,945 - INFO - 
Número de personas que fuman son fumadoras: 88341
2025-04-12 12:18:38,962 - INFO - 
Q1: 11.0, Q3: 24.0, IQR: 13.0
2025-04-12 12:18:38,963 - INFO - Límite inferior ajustado: 1
2025-04-12 12:18:38,964 - INFO - Límite superior: 43.5
2025-04-12 12:18:38,964 - INFO - Número de valores atípicos: 0


Age verification

In [386]:
age_max = df["Age"].max()
age_min = df["Age"].min()
logging.info(f"\nLa edad maxima es:  {age_max}")
logging.info(f"\nLa edad minima es: {age_min}")

2025-04-12 12:18:38,970 - INFO - 
La edad maxima es:  85
2025-04-12 12:18:38,971 - INFO - 
La edad minima es: 20


Map of columns

In [387]:
df_selected = df[["Age","Country","Lung_Cancer_Prevalence_Rate","Smoker","Years_of_Smoking","Cigarettes_per_Day","Passive_Smoker","Lung_Cancer_Diagnosis","Healthcare_Access","Early_Detection","Survival_Years","Developed_or_Developing","Mortality_Rate","Annual_Lung_Cancer_Deaths","Air_Pollution_Exposure","Occupational_Exposure","Indoor_Pollution","Family_History","Treatment_Type","Cancer_Stage"]]
logging.info(f"\n{df_selected.info()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220632 entries, 0 to 220631
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Age                          220632 non-null  int64  
 1   Country                      220632 non-null  object 
 2   Lung_Cancer_Prevalence_Rate  220632 non-null  float64
 3   Smoker                       220632 non-null  object 
 4   Years_of_Smoking             220632 non-null  int64  
 5   Cigarettes_per_Day           220632 non-null  int64  
 6   Passive_Smoker               220632 non-null  object 
 7   Lung_Cancer_Diagnosis        220632 non-null  object 
 8   Healthcare_Access            220632 non-null  object 
 9   Early_Detection              220632 non-null  object 
 10  Survival_Years               220632 non-null  int64  
 11  Developed_or_Developing      220632 non-null  object 
 12  Mortality_Rate               220632 non-null  float64
 13 

Cleaned null 

In [388]:
df_selected.fillna("any", inplace=True)
df_selected.isnull().sum()

/tmp/ipykernel_323468/3851637230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.fillna("any", inplace=True)


Age                            0
Country                        0
Lung_Cancer_Prevalence_Rate    0
Smoker                         0
Years_of_Smoking               0
Cigarettes_per_Day             0
Passive_Smoker                 0
Lung_Cancer_Diagnosis          0
Healthcare_Access              0
Early_Detection                0
Survival_Years                 0
Developed_or_Developing        0
Mortality_Rate                 0
Annual_Lung_Cancer_Deaths      0
Air_Pollution_Exposure         0
Occupational_Exposure          0
Indoor_Pollution               0
Family_History                 0
Treatment_Type                 0
Cancer_Stage                   0
dtype: int64

To boleans

In [389]:
df_selected["Early_Detection"] = df_selected["Early_Detection"].map({"Yes": True, "No": False})
df_selected["Smoker"] = df_selected["Smoker"].map({"Yes": True, "No": False})
df_selected["Passive_Smoker"] = df_selected["Passive_Smoker"].map({"Yes": True, "No": False})
df_selected["Lung_Cancer_Diagnosis"] = df_selected["Lung_Cancer_Diagnosis"].map({"Yes": True, "No": False})
df_selected["Occupational_Exposure"] = df_selected["Occupational_Exposure"].map({"Yes": True, "No": False})
df_selected["Indoor_Pollution"] = df_selected["Indoor_Pollution"].map({"Yes": True, "No": False})
df_selected["Family_History"] = df_selected["Family_History"].map({"Yes": True, "No": False})
logging.info(df_selected.info)

2025-04-12 12:18:39,421 - INFO - <bound method DataFrame.info of         Age       Country  Lung_Cancer_Prevalence_Rate  Smoker  \
0        80         China                         2.44    True   
1        53          Iran                         2.10   False   
2        47        Mexico                         1.11    True   
3        39     Indonesia                         0.75   False   
4        44  South Africa                         2.44   False   
...     ...           ...                          ...     ...   
220627   55      Thailand                         0.70    True   
220628   66  South Africa                         0.78   False   
220629   46       Vietnam                         1.16   False   
220630   36      Ethiopia                         1.06   False   
220631   60       Germany                         1.07   False   

        Years_of_Smoking  Cigarettes_per_Day  Passive_Smoker  \
0                     30                  29           False   
1             

/tmp/ipykernel_323468/1341390526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["Early_Detection"] = df_selected["Early_Detection"].map({"Yes": True, "No": False})
/tmp/ipykernel_323468/1341390526.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["Smoker"] = df_selected["Smoker"].map({"Yes": True, "No": False})
/tmp/ipykernel_323468/1341390526.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

## Load to postgress

In [390]:
importlib.reload(pg)
pg.crear_usuario_y_bd()

conn = pg.conectar_etl()
pg.crear_tabla_lung_cancer_post(conn)

El usuario 'etl_user' ya existe.
La base de datos 'lung_cancer_database' ya existe.
Tabla 'lung_cancer_post' verificada/creada.


In [391]:
pg.insertar_datos_lung_cancer_post(conn, df_selected)

Los datos ya están insertados. No se duplicaron registros.


In [392]:
df_post = pg.leer_datos_lung_cancer_post(conn)
print(df_post)

/mnt/d/Universidad/ETL/Proyecto_airflow/notebook/../config_documents/postgres_setup.py:147: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM lung_cancer_post", conn)


Se leyeron 220632 registros de 'lung_cancer_post'
            id  age       country  lung_cancer_prevalence_rate  smoker  \
0            1   80         China                         2.44    True   
1            2   53          Iran                         2.10   False   
2            3   47        Mexico                         1.11    True   
3            4   39     Indonesia                         0.75   False   
4            5   44  South Africa                         2.44   False   
...        ...  ...           ...                          ...     ...   
220627  220628   55      Thailand                         0.70    True   
220628  220629   66  South Africa                         0.78   False   
220629  220630   46       Vietnam                         1.16   False   
220630  220631   36      Ethiopia                         1.06   False   
220631  220632   60       Germany                         1.07   False   

        years_of_smoking  cigarettes_per_day  passive_smoker 

In [393]:
print(df_post.columns.tolist())

['id', 'age', 'country', 'lung_cancer_prevalence_rate', 'smoker', 'years_of_smoking', 'cigarettes_per_day', 'passive_smoker', 'lung_cancer_diagnosis', 'healthcare_access', 'early_detection', 'survival_years', 'developed_or_developing', 'mortality_rate', 'annual_lung_cancer_deaths', 'air_pollution_exposure', 'occupational_exposure', 'indoor_pollution', 'family_history', 'treatment_type', 'cancer_stage']


In [394]:
cur = conn.cursor()
cur.execute("""
DROP TABLE IF EXISTS 
    dim_diagnostico,
    dim_cancer_stage,
    dim_tratamiento,
    dim_historia_familiar,
    dim_fumador,
    dim_exposicion,
    dim_salud,
    dim_pais
CASCADE;
""")

In [395]:
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS dim_pais (
    id_pais SERIAL PRIMARY KEY,
    country VARCHAR(100),
    developed_or_developing VARCHAR(50),
    annual_deaths FLOAT
);
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS dim_salud (
        id_salud SERIAL PRIMARY KEY,
        healthcare_access VARCHAR(50),
        early_detection BOOLEAN,
        treatment_type VARCHAR(50),
        cancer_stage VARCHAR(50)
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS dim_fumador (
        id_fumador SERIAL PRIMARY KEY,
        fumador BOOLEAN,
        fumador_pasivo BOOLEAN,
        anios_fumando INT,
        cigarrillos_por_dia INT
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS dim_exposicion (
        id_exposicion SERIAL PRIMARY KEY,
        air_pollution_exposure VARCHAR(50),
        occupational_exposure BOOLEAN,
        indoor_pollution BOOLEAN,
        family_history BOOLEAN
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS dim_diagnostico (
        id_diagnostico SERIAL PRIMARY KEY,
        descripcion BOOLEAN UNIQUE
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS dim_cancer_stage (
        id_stage SERIAL PRIMARY KEY,
        descripcion VARCHAR(50) UNIQUE
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS dim_tratamiento (
        id_tratamiento SERIAL PRIMARY KEY,
        descripcion VARCHAR(50) UNIQUE
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS dim_historia_familiar (
        id_historia SERIAL PRIMARY KEY,
        descripcion BOOLEAN UNIQUE
    );
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS hechos_persona (
    id_persona SERIAL PRIMARY KEY,
    edad INT,
    tasa_prevalencia FLOAT,
    tasa_mortalidad FLOAT,
    survival_years INT,
    
    id_fumador INT REFERENCES dim_fumador(id_fumador),
    id_pais INT REFERENCES dim_pais(id_pais),
    id_salud INT REFERENCES dim_salud(id_salud),
    id_exposicion INT REFERENCES dim_exposicion(id_exposicion),
    id_diagnostico INT REFERENCES dim_diagnostico(id_diagnostico),
    id_stage INT REFERENCES dim_cancer_stage(id_stage),
    id_tratamiento INT REFERENCES dim_tratamiento(id_tratamiento),
    id_historia INT REFERENCES dim_historia_familiar(id_historia)
);
""")

conn.commit()
cur.close()
conn.close()
print("✅ Tablas de dimensiones creadas correctamente.")

✅ Tablas de dimensiones creadas correctamente.
